In [2373]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
with open('/content/drive/My Drive/AgrellA-Domd.xml', 'r', encoding='utf-8') as f:
    xml = f.read()

In [2375]:
import re
from lxml import etree
import html as html_
!pip install titlecase
from titlecase import titlecase
!pip install googletrans
from googletrans import Translator
translator = Translator()
import csv
import json
import datetime
from xml.etree import ElementTree as ET

In [2376]:
# replace soft hyphen by true hyphen
xml = re.sub(r'\xad', '-', xml)

# remove some tags
xml = re.sub(r'</?lb[^<>]*?>', '', xml)
xml = re.sub(r'</?placeName[^<>]*?>', '', xml)
xml = re.sub(r'</?emph[^<>]*?>', '', xml)
xml = re.sub(r'</?orgName[^<>]*?>', '', xml)
xml = re.sub(r'</?said[^<>]*?>', '', xml)
for i in range(10):
    print(i)
    xml = re.sub(r'(<p>.*?)(<persName[^><]*?>)(.*?)(</persName>)(.*?</p>)', r'\1\3\5', xml, 0, re.DOTALL)
    xml = re.sub(r'(<p>.*?)(<title>)(.*?)(</title>)(.*?</p>)', r'\1\3\5', xml, 0, re.DOTALL)
xml = re.sub(r'(<p>.*?)(<title>)(.*?)(</title>)(.*?</p>)', r'\1\3\5', xml, 0, re.DOTALL)

0
1
2
3
4
5
6
7
8
9


Define schema:

In [0]:
# schema = 'http://www.tei-c.org/ns/1.0'
# schema_with_brackets = '{' + schema + '}'

Let's replace "\<idno type="dracor">swe000044\</tei:idno>" with "\<idno type="dracor">swe000044\</idno>" (so that etree could parse the file)

In [0]:
xml = re.sub(r'(<idno type="dracor">)([^<]*?)</tei:idno>', r'\1\2</idno>', xml)

We will remove the comments as we don't save such information

In [0]:
parser = etree.XMLParser(remove_comments=True, remove_blank_text=True)
root = etree.fromstring(xml, parser=parser)

In [0]:
for tag in root.iter():
    if '}' in tag.tag:
        tag.tag = tag.tag.split('}', 1)[1]  # strip all namespaces

Define prefix

In [0]:
prefix = '{http://www.w3.org/XML/1998/namespace}'

Let's add 'xml-stylesheet' tag and set lang in TEI tag:

In [0]:
root.addprevious(etree.PI('xml-stylesheet', 'type="text/css" href="../css/tei.css"'))
root.set(prefix + "lang", "sv")

In [0]:
teiHeader, text = root.getchildren()

In [0]:
fileDesc, encodingDesc, profileDesc = teiHeader.getchildren()

In [0]:
titleStmt, editionStmt, publicationStmt, sourceDesc = fileDesc.getchildren()

In [0]:
titleStmt_children = titleStmt.getchildren()

sub_title = None
if titleStmt_children[1].tag == 'title':
    # there is a sub-title
    main_title, sub_title, author, *others = titleStmt_children
else:
    # there is no sub-title
    main_title, author, *others = titleStmt_children

Let's add xml:lang attribute to the title(s):

In [0]:
main_title.set(prefix + "lang", "sv")
if sub_title is not None:
    sub_title.set(prefix + "lang", "sv")

Let's add translations (English) of titles to the 'titleStmt':

In [0]:
main_title_en = etree.Element('title', type='main')

# set xml:lang
main_title_en.set(prefix + "lang", "en")

en_translation = translator.translate(main_title.text, src='sv', dest='en').text
# capitalize non-functional words
main_title_en.text = titlecase(en_translation).strip()
# insert after main_title on swedish
titleStmt.insert(1, main_title_en)

In [0]:
if sub_title is not None:
    sub_title_en = etree.Element('title', type='sub')

    # set xml:lang
    sub_title_en.set(prefix + "lang", "en")

    en_translation = translator.translate(sub_title.text, src='sv', dest='en').text
    # capitalize non-functional words
    sub_title_en.text = titlecase(en_translation).strip()
    # insert after sub_title on swedish
    titleStmt.insert(3, sub_title_en)

In [0]:
forename, surname = author.getchildren()[0].getchildren()

In [0]:
forename, surname = forename.text, surname.text

In [0]:
name = forename + ' ' + surname

In [0]:
# with open('authors_to_wikidata_urls.json', 'r', encoding='utf-8') as f:
#     authors_to_wikidata_urls = json.load(f)

authors_to_wikidata_urls = {}

with open('/content/drive/My Drive/SweDraCor_authors_Wikidata_ids.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader, None) # skip header
    for row in reader:
        authors_to_wikidata_urls[row[0]] = row[1]

with open('authors_to_wikidata_urls.json', 'w', encoding='utf-8') as f:
    json.dump(authors_to_wikidata_urls, f)

In [0]:
# with open('plays_to_wikidata_urls.json', 'r', encoding='utf-8') as f:
#     plays_to_wikidata_urls = json.load(f)

plays_to_wikidata_urls = {}

with open('/content/drive/My Drive/SweDraCor_plays_Wikidata_ids.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader, None) # skip header
    for row in reader:
        plays_to_wikidata_urls[row[1]] = row[3]

with open('plays_to_wikidata_urls.json', 'w', encoding='utf-8') as f:
    json.dump(plays_to_wikidata_urls, f)

In [0]:
# authors_to_wikidata_urls[name].split('/')[-1], plays_to_wikidata_urls[main_title.text].split('/')[-1]

In [0]:
# remove old author
titleStmt.remove(author)

# add new author
author = etree.Element('author', key='wikidata:' + authors_to_wikidata_urls[name].split('/')[-1])
author.text = surname + ', ' + forename
titleStmt.append(author)

Let's delete reference information (sponsors/funders/editors/principals/encoders):

In [0]:
for other in others:
    titleStmt.remove(other)

Let's remove editionStmt:

In [0]:
fileDesc.remove(editionStmt)

In [0]:
publisher, address, first_idno, second_idno, availability = publicationStmt.getchildren()

In [0]:
publisher.text = 'RusDraCor'
publisher.set(prefix + "id", "RusDraCor")

Let's remove address:

In [0]:
publicationStmt.remove(address)

In [0]:
first_idno.set('type', 'URL')
first_idno.text = 'https://dracor.org/swe/' + surname.lower() + '-' + '-'.join(main_title.text.lower().split())

In [0]:
second_idno.set(prefix + "base", "https://dracor.org/id/")

In [0]:
del availability.attrib['status']

In [0]:
licence = availability.getchildren()[0]

In [0]:
del licence.attrib['target']

In [0]:
for child in licence.getchildren():
    licence.remove(child)

In [0]:
ab = etree.Element('ab')
ab.text = 'CC0'
licence.append(ab)

In [0]:
ref = etree.Element('ref', target='https://creativecommons.org/publicdomain/zero/1.0/')
ref.text = 'Licence'
licence.append(ref)

In [0]:
idno = etree.Element('idno', type='wikidata')
idno.set(prefix + "base", "https://www.wikidata.org/entity/")
idno.text = plays_to_wikidata_urls[main_title.text].split('/')[-1]

In [0]:
publicationStmt.append(idno)

In [0]:
biblStruct = sourceDesc.getchildren()[0]

In [0]:
sourceDesc.remove(biblStruct)

In [0]:
title, idno, imprint = biblStruct.getchildren()[1].getchildren()[1:]

In [0]:
url = idno.text
title_text = title.text

In [0]:
pubPlace, publisher, date, respStmt, *others = imprint.getchildren()

In [0]:
when = date.attrib['when']

In [0]:
bibl = etree.Element('bibl', type='digitalSource')
sourceDesc.append(bibl)

In [0]:
name = etree.Element('name')
name.text = ''
bibl.append(name)

In [0]:
idno = etree.Element('idno', type='URL')
idno.text = url
bibl.append(idno)

In [0]:
availability = etree.Element('availability', status='free')
p = etree.Element('p')
p.text = 'In the public domain.'
availability.append(p)
bibl.append(availability)

In [0]:
inner_bibl = etree.Element('bibl', type='originalSource')
bibl.append(inner_bibl)

In [0]:
title = etree.Element('title')
title.text = title_text + ', ' + pubPlace.text + ', ' + publisher.text + ', ' + respStmt.getchildren()[-1].text + ', ' + when + ' ' + '(' + url + ')'
inner_bibl.append(title)

In [0]:
date = etree.Element('date', type='print', when=when)
date.text = '"' + when + '"' + ' ' + '(' + url + ')'
inner_bibl.append(date)

In [0]:
date = etree.Element('date', type='premiere')
inner_bibl.append(date)

In [0]:
date = etree.Element('date', type='written')
inner_bibl.append(date)

Let's remove encodingDesc

In [0]:
teiHeader.remove(encodingDesc)

profileDesc

In [0]:
for child in profileDesc:
    profileDesc.remove(child)

In [0]:
def add_particDesc_to_profileDesc(profileDesc):
    # particDesc to profileDesc
    particDesc = etree.Element('particDesc')
    profileDesc.append(particDesc)
    return particDesc

In [0]:
def add_listPerson_to_particDesc(particDesc):
    # listPerson to particDesc
    listPerson = etree.Element('listPerson')
    particDesc.append(listPerson)
    return listPerson

In [0]:
particDesc = add_particDesc_to_profileDesc(profileDesc)

In [0]:
listPerson = add_listPerson_to_particDesc(particDesc)

In [0]:
def add_textClass_to_profileDesc(profileDesc):
    # textClass to profileDesc
    textClass = etree.Element('textClass')
    profileDesc.append(textClass)
    return textClass

In [0]:
textClass = add_textClass_to_profileDesc(profileDesc)

In [0]:
def add_keywords_to_textClass(textClass):
    # keywords to textClass
    keywords = etree.Element('keywords')
    textClass.append(keywords)
    return keywords

In [0]:
keywords = add_keywords_to_textClass(textClass)

In [0]:
def add_term_to_keywords(keywords):
    term = etree.Element('term', type='genreTitle', subtype='')
    term.text = ''
    keywords.append(term)
    return term

In [0]:
term = add_term_to_keywords(keywords)

revisionDesc

In [0]:
revisionDesc = etree.Element('revisionDesc')
teiHeader.append(revisionDesc)
listChange = etree.Element('listChange')
revisionDesc.append(listChange)
change = etree.Element('change', when=datetime.datetime.now().strftime("%Y-%m-%d"))
change.text = "(%s) convert from source" % ('eg') # replace if necessary
listChange.append(change)

text

removing anchors (?)

In [0]:
for anchor in text.xpath('//anchor'):
    parent = anchor.getparent()
    parent.remove(anchor)

delete xml:id attribute of text

In [0]:
del text.attrib[prefix + 'id']

text children

In [0]:
front, body, back = text.getchildren()

In [0]:
castList = front.getchildren()[-1]

In [0]:
castList_children = castList.getchildren()

In [0]:
for child_index, child in enumerate(castList_children):
    if child.tag == 'addSpan':
        break

In [0]:
# remove addSpan
castList.remove(castList_children[child_index])

# we don't save them

# remove castItem type="list"
castList.remove(castList_children[child_index + 1])

In [0]:
castList_children = castList.getchildren()

remove children of 'front'

In [0]:
for child in front.getchildren():
    front.remove(child)

In [0]:
def add_docTitle_to_front(front):
    # docTitle to front
    docTitle = etree.Element('docTitle')
    front.append(docTitle)
    return docTitle

In [0]:
docTitle = add_docTitle_to_front(front)

In [0]:
def add_titlePart_main_to_docTitle(docTitle, title):
    # titlePart to docTitle
    titlePart = etree.Element('titlePart', type='main')
    titlePart.text = title
    docTitle.append(titlePart)
    return titlePart

In [0]:
def add_titlePart_sub_to_docTitle(docTitle, title):
    # titlePart to docTitle
    titlePart = etree.Element('titlePart', type='sub')
    titlePart.text = title
    docTitle.append(titlePart)
    return titlePart

In [2453]:
add_titlePart_main_to_docTitle(docTitle, main_title.text)

<Element titlePart at 0x7f1eb87fc808>

In [2454]:
add_titlePart_sub_to_docTitle(docTitle, sub_title.text)

<Element titlePart at 0x7f1eb87f0fc8>

In [0]:
def inner_xml(element):
    return (element.text or '') + ''.join(ElementTree.tostring(e, 'unicode') for e in element)

Change castList:

In [0]:
id_to_ref = {}
ref_to_id = {}
for castItem in castList_children[1:]:
    roleDesc = ''
    castItem_children = castItem.getchildren()
    if len(castItem_children) == 2:
        role, roleDesc = castItem_children
    else:
        role = castItem_children[0]
    ref = role.attrib.values()[0][2:]
    id_ = role.attrib.values()[0]
    id_to_ref[id_] = ref
    ref_to_id[ref] = id_
    full_text = ''
    for part_of_text in castItem.itertext():
        full_text += part_of_text
    castItem.remove(role)
    if len(castItem_children) == 2:
        castItem.remove(roleDesc)
    castItem.text = re.sub(r'\s{2,}', '', full_text.strip())

append castList to body

In [0]:
body.insert(0, castList)

We will remove \<pb> tag (though may save if necessary).

In [0]:
for anchor in text.xpath('//pb'):
    parent = anchor.getparent()
    parent.remove(anchor)

Also we will delete \<lb/> tag because we don't encode it

In [0]:
_, *divs, byline = body.getchildren()

In [0]:
def recursive_clean(div):
    if div.tag == 'div':
        # delete n attrib
        del div.attrib['n']
    elif div.tag == 'sp':
        tmp = ['#' + elem[1 + 2 * (elem.startswith('#sp')):] for elem in div.attrib['who'].split()]
        for i, elem in enumerate(tmp):
            tmp[i] = re.sub(r'([A-ZÅÄÖ][a-zåäö]+)(?=[A-ZÅÄÖ])', r'\1 ', elem[1:]).lower()
            tmp[i] = '#' + tmp[i].replace(' ', '_')
        div.attrib['who'] = ' '.join(tmp)
    for tag in div.getchildren():
        full_text = inner_xml(tag)
        if tag.tag not in ['p', 'l', 'stage']:
            for tag_child in tag.getchildren():
                if tag_child.tag in ['lb', 'persName', 'placeName']:
                    # delete lb, persName, placeName tags
                    tag.remove(tag_child)
                elif tag_child.tag not in ['stage']:
                    recursive_clean(tag_child)

#         print(tag.text, tag.tag)
        if tag.text:
            tag.text = re.sub(r'\s{2,}', ' ', tag.text).strip()
            tag.text = re.sub('\n', ' ', tag.text)
        recursive_clean(tag)
    if div.text:
        div.text = re.sub(r'\s{2,}', ' ', div.text).strip()
        div.text = re.sub('\n', ' ', div.text)

In [0]:
for div in divs:
    recursive_clean(div)

In [0]:
date = byline.getchildren()[0]

In [0]:
written_date = bibl.getchildren()[-1].getchildren()[-1]
written_date.set('when', date.text)
written_date.text = date.text

In [0]:
body.remove(byline)

In [0]:
div = back.getchildren()[0]

In [0]:
tmp_listPerson, listOrg, *_ = div.getchildren()

In [0]:
cast_listPerson, noncast_listPerson, _ = tmp_listPerson.getchildren()

In [0]:
list_of_speakers = []
set_of_speakers = set()
for sp in root.xpath('//sp'):
    whos = sp.attrib['who'].split()
    for who in whos:
        if who not in set_of_speakers:
            set_of_speakers.add(sp.attrib['who'])
            list_of_speakers.append(sp.attrib['who'])

In [0]:
list_of_speakers = [elem[1:] for elem in list_of_speakers]

In [2470]:
list_of_speakers

['ivar',
 'valborg',
 'olof',
 'lisen',
 'hilma',
 'professorn',
 'gertrud',
 'björnklo',
 'fru_hillner',
 'direktör_hillner',
 'assessorn',
 'fröken_bonn',
 'farmor',
 'mantell',
 'anna']

In [0]:
xml_id_to_sex = {}
xml_id_to_fullname = {}
xml_id_to_type_of_person = {}
for child in cast_listPerson.getchildren():
    if child.tag in [elem for elem in ['person', 'personGrp']]:
        xml_id = child.attrib[prefix + 'id']
        xml_id = re.sub(r'([A-ZÅÄÖ][a-zåäö]+)(?=[A-ZÅÄÖ])', r'\1 ', xml_id).lower()
        xml_id = xml_id.replace(' ', '_')
        if xml_id in list_of_speakers:
            if child.tag == 'person':
                xml_id_to_type_of_person[xml_id] = 'person'
            else:
                xml_id_to_type_of_person[xml_id] = 'personGrp'
            xml_id_to_sex[xml_id] = child.attrib['sex'].upper()
            persName, *_ = child.getchildren()
            full_text = ''
            for part_of_text in persName.itertext():
                full_text += part_of_text + ' '
#             full_text = inner_xml(persName)
            full_text = full_text.strip()
            xml_id_to_fullname[xml_id] = full_text

In [2472]:
set(list_of_speakers) ^ set(xml_id_to_sex.keys())

set()

In [2473]:
set(list_of_speakers) ^ set(xml_id_to_fullname.keys())

set()

In [0]:
for xml_id in list_of_speakers:
    if xml_id_to_type_of_person[xml_id] == 'person':
        person = etree.Element('person')
        person.set(prefix + "id", xml_id)
        person.set('sex', xml_id_to_sex[xml_id])
        persName = etree.Element('persName')
        persName.text = xml_id_to_fullname[xml_id]
        person.append(persName)
        listPerson.append(person)

In [0]:
text.remove(back)

In [0]:
result = etree.tostring(root.getroottree(), pretty_print=True, xml_declaration=True, encoding="UTF-8").decode('utf-8')
result = html_.unescape(result)
result = re.sub(r'<fw type="pageNum">([0-9]+)</fw>', r'<pb n="\1"/>', result)
result = re.sub(r'([^\s])(<[^/])', r'\1 \2', result)
for i in range(10):
#     print(i)
    result = re.sub(r'(<p(?: rend=\"[^><]*?\")?>(?:(?!</p>).)*?)\s{2,}((?:(?!</p>).)*?</p>)', r'\1 \2', result, 0, re.DOTALL)

result = re.sub(r'(</[^>]+>) (</[^>]+>)', r'\1\2', result)
result = re.sub(r'/>([^\s])', r'/> \1', result)
result = re.sub(r'\s+</stage>', r'</stage>', result)
result = re.sub(r'\( <', r'(<', result)
# result = re.sub(r'> <', r'><', result)
result = re.sub(r',([^\s])', r', \1', result)
result = re.sub(r' ,', r',', result)
result = re.sub(r'\s+([^<\s])', r' \1', result)

result = re.sub(r'\n\s+([^><]*?)</stage>', r' \1</stage>', result)
print(result)

In [0]:
# parser =  etree.XMLParser(remove_comments=True, remove_blank_text=True)
# root = etree.fromstring("""<sp who="#spGertrud">
#              <speaker>Gertrud.</speaker>
#              <p>Ja, inte sant! Jag tror att jag aldrig sett en
#                så vacker kläd— — <stage>(hejdar sig, stampar i golfvet).</stage>
#                         </p>
#            </sp>""", parser=parser)

In [0]:
with open(first_idno.text.split('/')[-1] + '.xml', 'w', encoding='utf-8') as f:
    f.write(result)